In [204]:
from qiskit.circuit import QuantumCircuit, QuantumRegister
import ffsim
import math
import numpy as np
from passive_extended_matchgate_simulator.raw_estimation import raw_estimate
from passive_extended_matchgate_simulator.utils import ucj_to_compatible, ucj_to_compatible_fully_reduced, calculate_trajectory_count, extract_circuit_data

norb = 3
nelec = (2, 2)
alpha_alpha_indices = [(p, p + 1) for p in range(norb - 1)]
alpha_beta_indices = [(p, p) for p in range(norb)]
qubits = QuantumRegister(2 * norb)
circuit = QuantumCircuit(qubits)
ucj_op = ffsim.random.random_ucj_op_spin_balanced(norb, 
                                                  interaction_pairs=(alpha_alpha_indices, alpha_beta_indices),
                                                  with_final_orbital_rotation=False,
                                                  diag_coulomb_mean=math.pi, 
                                                  diag_coulomb_scale=.1, 
                                                  diag_coulomb_normal=True)
circuit.append(ffsim.qiskit.PrepareHartreeFockJW(norb, nelec), qubits)
circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(ucj_op), qubits)

statevec = ffsim.qiskit.final_state_vector(circuit)
ffsim_probs = np.abs(statevec.vec)**2
bitstrings = ffsim.addresses_to_strings(range(len(ffsim_probs)), norb, nelec)
ffsim_p_and_b = list(zip(ffsim_probs, bitstrings))

epsilon = .1
delta = .01
p = 1

compatible = ucj_to_compatible_fully_reduced(circuit)

In [205]:
ffsim_p_and_b

[(np.float64(0.6442219119696962), 27),
 (np.float64(0.022762394851997304), 43),
 (np.float64(0.03473879384812939), 51),
 (np.float64(0.022762394851997297), 29),
 (np.float64(0.14485083441896876), 45),
 (np.float64(0.011151091600629656), 53),
 (np.float64(0.0347387938481294), 30),
 (np.float64(0.011151091600629673), 46),
 (np.float64(0.07362269300982753), 54)]

In [206]:
raw_estimate_values = []
for b in bitstrings:
    raw_estimate_values.append(raw_estimate(circuit=compatible,outcome_state=b, epsilon=epsilon, delta=delta, p=p))
raw_estimate_values

[0.6459155674078316,
 0.023202671685870948,
 0.033110257898519686,
 0.022653542295743054,
 0.1426189878095225,
 0.01029127995516482,
 0.03407877553949969,
 0.01106443726654977,
 0.07367986081048011]

In [207]:
diff = np.abs(ffsim_probs - raw_estimate_values)
print("max difference", max(diff))
data = extract_circuit_data(compatible)
extent = data[0]
t = calculate_trajectory_count(epsilon, delta, extent, p)
print("trajectory count", t)
diff


max difference 0.002231846609446253
trajectory count 921069


array([1.69365544e-03, 4.40276834e-04, 1.62853595e-03, 1.08852556e-04,
       2.23184661e-03, 8.59811645e-04, 6.60018309e-04, 8.66543341e-05,
       5.71678007e-05])